# Part 1: Forward Mode Automatic Differentiation

Forward mode AD can simply be implemented by defining a class to represent [dual numbers](https://en.wikipedia.org/wiki/Dual_number) which hold the value and its derivative. The following skeleton defines a dual number and implements multiplication. 

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it? 
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [2]:
import math

class DualNumber:
    def __init__(self, value, dvalue):
        self.value = value
        self.dvalue = dvalue

    def __repr__(self):
        return repr(self.value) + ' + ' + repr(self.dvalue)  + "ε"
    
    def __str__(self):
        return str(self.value) + " + " + str(self.dvalue) + "ε"

    def __mul__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.value * other.value,
                self.dvalue * other.value + other.dvalue * self.value)

        return DualNumber(self.value * other, self.dvalue * other)
    
    __rmul__ = __mul__

    def __add__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.value + other.value, self.dvalue + other.dvalue)

        return DualNumber(self.value + other, self.dvalue)

    __radd__ = __add__


    def __sub__(self, other):
        return DualNumber(self.value - other.value, self.dvalue - other.dvalue)

    
    def __truediv__(self, other):
        try: # only if other.value != 0
            x = 1/other.value
        except ZeroDivisionError as err:
            print('Handling run-time error:', err)

        return DualNumber(self.value/other.value, (self.dvalue*other.value-self.value*other.dvalue)/other.value**2)
    
    def __pow__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.value**other.value,
                          self.value**other.value *other.dvalue*math.log(self.value) 
                          + other.value*self.dvalue*self.value**(other.value-1))
            
        return DualNumber(self.value**other, self.dvalue*other*self.value**(other - 1))


$$
(a + b\epsilon) + (c + d\epsilon) = (a + c) + (b + d)\epsilon \\
(a + b\epsilon) - (c + d\epsilon) = (a - c) + (b - d)\epsilon \\
(a + b\epsilon) * (c + d\epsilon) = (ac) + (ad - bc)\epsilon \\
\frac{(a + b\epsilon)}{(c + d\epsilon)} = \frac{a}{c} + \frac{(bc - ad)}{c^2}\epsilon \\
(a + b\epsilon)^y =  a^y + bya^{y-1}\epsilon \\
(a + b\epsilon)^{c+d\epsilon} = a^c + (a^c d In(a) + c a ^{c-1}b)\epsilon 
$$

In [3]:
# Tests

DualNumber(1,0) + DualNumber(1,0) / DualNumber(1,0) - DualNumber(1,0)**DualNumber(1,0)


1.0 + 0.0ε

## Implementing math functions

We also need to implement some core math functions. Here's the sine function for a dual number:

In [4]:
def sin(x):
    return DualNumber(math.sin(x.value), math.cos(x.value)*x.dvalue)

__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

$$
f(a) = f(a) + f'(a)\epsilon \\
cos(\epsilon) = 1 \\
sin(\epsilon) = \epsilon \\
tan(\epsilon) = \epsilon \\
sin(a + b\epsilon) = sin(a) + bcos(a) \epsilon \\
cost(a + b\epsilon) = cos(a) - bsin(a)\epsilon \\
tan(a + b\epsilon) = tan(a) + \frac{b}{cos^2(a)}\epsilon
$$

In [5]:
# implement additional math functions on dual numbers

def cos(x):
    # YOUR CODE HERE
    return DualNumber(math.cos(x.value), -math.sin(x.value)*x.dvalue)

def tan(x):
    # YOUR CODE HERE
    return DualNumber(math.tan(x.value), x.dvalue/(math.cos(x.value)**2))

def exp(x):
    # YOUR CODE HERE
    return DualNumber(math.exp(x.value), math.exp(x.value) * x.dvalue)

In [6]:
# Tests
assert cos(DualNumber(0,0)).value == 1
assert tan(DualNumber(0,0)).value == 0
assert exp(DualNumber(0,0)).value == 1


## Time to try it out

We're now in a position to try our implementation.

__Task:__ 

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point. 

In [7]:
# YOUR CODE HERE
def calZ(x, y):
    return x * y + math.sin(x)

x = 0.5
y = 4.2
calZ(x, y)


2.579425538604203

In [8]:
a = DualNumber(x, 1)
b = DualNumber(y, 0) # w.r.t x, so it's 0

a * b + sin(a)

2.579425538604203 + 5.077582561890373ε

__Task__: Differentiate the above function with respect to $x$ and write the symbolic derivatives in the following box. Verify the result computed above is correct by plugging-in the values into your symbolic gradient expression.

$$
\frac{\partial z}{\partial x} = y + cos(x) \\
\frac{\partial z}{\partial y} = x \\
$$

In [9]:
def derivativeX(x, y):
    return y + math.cos(x)

derivativeX(x, y)


5.077582561890373

__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above) and store the derivative in the variable `dzdy` (just the derivative, not the Dual Number). Verify by hand that the result is correct.

In [15]:
# YOUR CODE HERE
a = DualNumber(x, 0) # w.r.t y, so it's 0
b = DualNumber(y, 1)

print(a * b + sin(a))

def derivativeY(x, y):
    return x

dzdy = derivativeY(x, y)

print('dz/dy:', dzdy)

2.579425538604203 + 0.5ε
dz/dy: 0.5


In [16]:
#Tests
assert dzdy
assert type(dzdy) == float


__Task:__ Finally, use the code block below to experiment and test the other math functions and methods you created.

In [12]:
def quadratic(x):
    return 5*x**2 + 4*x + 1

x1 = 2
xdual = DualNumber(x1, 1)


In [13]:
xdual, quadratic(x1), quadratic(xdual)


(2 + 1ε, 29, 29 + 24ε)

In [14]:
def derivativeQuaX(x):
    return 10 * x + 4

derivativeQuaX(x1)


24